__Notification__: The raw data from PubChem database was somewhat have lot of error and duplicate, therefore, we need to clean it first before.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

import os
import psycopg2

from dotenv import load_dotenv
import logging

#MACCS
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import RDLogger  

## Preparing

### Class definition

In [2]:
logging.basicConfig(filename='../../output/logs/100mil-hdac2-preprocessing.txt', level=logging.INFO)

In [3]:
class PubchemDAO:
    """
    Data access object of the PubChem raw data, the table include two columns: pubchem_compounds_cid, and pubchem_openeye_iso_smiles
    """
    
    SELECT_DATA_BETWEEN_CID = """SELECT * FROM 
        compound_screen_10mil WHERE pubchem_compound_cid >= %s and pubchem_compound_cid <= %s;"""
    MAX_CID_QUERY = "SELECT MAX(pubchem_compound_cid) FROM compound_screen_10mil"
    MIN_CID_QUERY = "SELECT MIN(pubchem_compound_cid) FROM compound_screen_10mil"
    
    max_cid = None
    min_cid = None
    connection = None
    
    def __init__(self):
        # Create connection
        load_dotenv("../env/.env")
        self.connection = psycopg2.connect(os.environ['DATABASE_URL'])
        # Get the max_cid and min_cid
        self.get_min_max_cid()
        logging.info(f"Min CID - Max CID: {self.min_cid} - {self.max_cid}")
    
    def get_min_max_cid(self):
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.MIN_CID_QUERY)
                self.min_cid = cursor.fetchall()[0][0]
                cursor.execute(self.MAX_CID_QUERY)
                self.max_cid = cursor.fetchall()[0][0]
        
    def get_data_between_cid(self, first_cid, second_cid):
        if(first_cid < self.min_cid or second_cid > self.max_cid):
            logging.info("Invalid cid!")
            return
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.SELECT_DATA_BETWEEN_CID, (first_cid, second_cid))
                data_list = cursor.fetchall()
                column_names = [desc[0] for desc in cursor.description]
                return pd.DataFrame(data_list, columns=column_names)

class PubchemScreeningDAO:  
    """
    Data access object of the data after preprocessed the PubChem database, 
        the table include 4 columns: id, cid, smiles, molecular_weight
    """  
    CREATE_PREPROCESSING_TABLE = """CREATE TABLE IF NOT EXISTS pubchem_compound_preprocessed (
        id SERIAL PRIMARY KEY,
        cid INTEGER UNIQUE NOT NULL, 
        smiles TEXT UNIQUE NOT NULL,
        molecular_weight decimal NOT NULL
    );
    CREATE INDEX IF NOT EXISTS idx_cid ON pubchem_compound_preprocessed (cid);
    """
    INSERT_COMPOUND = "INSERT INTO pubchem_compound_preprocessed (cid, smiles, molecular_weight) VALUES (%s, %s, %s);"
    SELECT_FIRST_DATA = "SELECT * FROM pubchem_compound_preprocessed LIMIT %s;"
    SELECT_DATA_BY_CID = """SELECT * FROM pubchem_compound_preprocessed WHERE cid = %s"""
    SELECT_NUMBERS_OF_ROWS = """SELECT COUNT(*) as row_count FROM pubchem_compound_preprocessed;"""
    MAX_CID_QUERY = "SELECT MAX(cid) FROM pubchem_compound_preprocessed"
    MIN_CID_QUERY = "SELECT MIN(cid) FROM pubchem_compound_preprocessed"
    
    connection = None
    
    def __init__(self):
        load_dotenv("../env/.env")
        self.connection = psycopg2.connect(os.environ['DATABASE_URL'])

    def create_tables(self):
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.CREATE_PREPROCESSING_TABLE)
    
    def read_first_rows(self, number_of_data):
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.SELECT_FIRST_1000_ALL_DATA, number_of_data)
                column_names = [desc[0] for desc in cursor.description]
                data_list = cursor.fetchall()
                return pd.DataFrame(data_list, columns=column_names)
    
    def insert(self, cid, smiles, molecular_weight):
        with self.connection:
            with self.connection.cursor() as cursor:
                try:
                    cursor.execute(self.INSERT_COMPOUND, (cid, smiles, molecular_weight))
                    return True
                except psycopg2.IntegrityError as error:
                    if 'duplicate key value violates unique constraint' in str(error):
                        logging.error(f"Skipping data with cid={cid} due to unique constraint violation.")
                    else:
                        logging.error(str(error))
                        raise  # Re-raise the exception if it's not related to unique constraint violation
                except Exception as e:
                    logging.error("An exception occurred: " + str(e))
        return False  # Return False if no successful insertion or if an exception occurs
    
    def get_number_of_rows(self):
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.SELECT_NUMBERS_OF_ROWS)
                nor = cursor.fetchall()
                return nor
            
    def get_min_max_cid(self):
        with self.connection:
            with self.connection.cursor() as cursor:
                cursor.execute(self.MIN_CID_QUERY)
                min_cid = cursor.fetchall()[0][0]
                cursor.execute(self.MAX_CID_QUERY)
                max_cid = cursor.fetchall()[0][0]
                return min_cid, max_cid

### Getting the train - test data

In [4]:
train_test_path = "../../data_for_modeling/train_test_data/HDAC2_train_test_data_final.xlsx"
train_dataset = pd.read_excel(train_test_path, sheet_name='train_dataset')
test_dataset = pd.read_excel(train_test_path, sheet_name='test_dataset')
validation_dataset = pd.read_excel(train_test_path, sheet_name='validation_dataset')

train_test_dataset = pd.concat([train_dataset, test_dataset, validation_dataset], axis=0)

In [5]:
print(len(train_dataset), len(test_dataset), len(validation_dataset), len(train_test_dataset))

2016 504 281 2801


In [6]:
train_test_dataset.head()

,CID,SMILES,AVG_IC50_nM,FINAL_LABEL,ZBG Classified
0,44138033,COC(=O)c1ccc(CO/N=C/c2ccc(/C=C/C(=O)NO)cc2)cc1,15800.0,Inactive,1
1,164616411,CC[C@H](NC(=O)C1CN(C)C1)c1ncc(-c2cc3ccccc3nc2O...,50000.0,Inactive,15
2,135357843,O=C(NO)c1ccc(Cn2nnnc2-c2cccs2)cc1,3698.0,Inactive,1
3,142506189,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(NCc3ccc(C(...,7173.0,Inactive,1
4,155537112,CN(C)c1ccc(C(=O)N(CC(=O)NCc2ccccc2)Cc2ccc(C(=O...,283.0,Inactive,4


## Checking and insert data into the preprocessing table

In [9]:
def check_error_fpts(check_dataset, smiles_column):
    """
    Checks for errors in MACCS fingerprint calculation.

    Args:
        check_dataset (pd.DataFrame): The dataset to be checked.
        smiles_column (str): The name of the SMILES column.

    Returns:
        pd.DataFrame: The rows with errors in MACCS fingerprint calculation.
    """
    result_df = pd.DataFrame(columns=check_dataset.columns)
    for index, row in check_dataset.iterrows():
        current_smiles = str(row[smiles_column]).strip()
        if current_smiles is not None and len(current_smiles) > 0:
            try:
                RDLogger.DisableLog('rdApp.info')
                mol = Chem.MolFromSmiles(current_smiles)
                if mol is not None:
                    result_df = pd.concat([result_df, row.to_frame().T], axis=0)  # Concatenate the current row                
                else:
                    logging.info("Could not interpret " + current_smiles + " to mol object!")
            except Exception as e:
                logging.error("An exception occurred at row " + str(index) + ": " + str(e))
                continue
    return result_df

def preprocess_dataset(working_dataset, train_test_dataset):
    """
    Preprocesses the dataset by removing duplicate SMILES and rows with errors in MACCS fingerprint calculation.

    Args:
        working_dataset (pd.DataFrame): The dataset to be preprocessed.
        train_test_dataset (pd.DataFrame): The training/testing dataset.

    Returns:
        pd.DataFrame: The preprocessed dataset.
    """
    #Filter
    logging.info("[+] Working dataset: " + str(len(working_dataset)))
    #Check for error smiles while encoding
    working_dataset = check_error_fpts(working_dataset, 'pubchem_openeye_iso_smiles')
    #Resert index
    working_dataset.reset_index(drop=True, inplace=True) 
    # Get the duplicate SMILES from the training/testing dataset.
    duplicate_smiles = working_dataset[working_dataset['pubchem_openeye_iso_smiles'].isin(train_test_dataset['SMILES'])]
    logging.info("[+] Duplicate with the train-test data: " + str(len(duplicate_smiles)))
    # Get the indices of duplicate smiles in test_working_dataset
    duplicate_indices = duplicate_smiles.index
    # Remove rows with duplicate SMILES from test_working_dataset
    working_dataset.drop(index=duplicate_indices, inplace=True)
    #Ending report
    logging.info("[+] Ending preprocessing: " + str(len(working_dataset)))
    return working_dataset

## Screening database preprocessing

In [10]:
pubchem_dao = PubchemDAO()
screening_dao = PubchemScreeningDAO()

### Test data preprocessing

In [9]:
test_working_dataset = pubchem_dao.get_data_between_cid(1, 10)
error_dataset = {
    test_working_dataset.columns[0]: [6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    test_working_dataset.columns[1]: ['O=C(NO)c1cnc(NC2(c3ccc(C(F)(F)F)c(F)c3)CC2)nc1', '', '4453', 'hellow rodl', 'Cc1cn(C)nc1CN1CCC(c2ccc(C(=O)Nc3ccccc3N)cc2)CC1', 'qwe', '####@@qds', None, ' ', '']
}
error_dataset = pd.DataFrame(error_dataset)
test_working_dataset = pd.concat([test_working_dataset, error_dataset], axis=0)

In [10]:
test_working_dataset = preprocess_dataset(working_dataset=test_working_dataset, train_test_dataset=train_test_dataset)

[09:56:24] SMILES Parse Error: syntax error while parsing: 4453
[09:56:24] SMILES Parse Error: Failed parsing SMILES '4453' for input: '4453'
[09:56:24] SMILES Parse Error: syntax error while parsing: hellow
[09:56:24] SMILES Parse Error: Failed parsing SMILES 'hellow' for input: 'hellow'
[09:56:24] SMILES Parse Error: syntax error while parsing: qwe
[09:56:24] SMILES Parse Error: Failed parsing SMILES 'qwe' for input: 'qwe'
[09:56:24] SMILES Parse Error: syntax error while parsing: ####@@qds
[09:56:24] SMILES Parse Error: Failed parsing SMILES '####@@qds' for input: '####@@qds'
[09:56:24] SMILES Parse Error: syntax error while parsing: None
[09:56:24] SMILES Parse Error: Failed parsing SMILES 'None' for input: 'None'


### Starting preprocessing

In [11]:
screening_dao.create_tables()

In [12]:
screening_dao.get_min_max_cid()

(1, 139999999)

In [14]:
# Choosing the starting CID to process by min_cid, and the final cid by max_cid.
# Steps: the number of precessed data for each batch
min_cid = 0
max_cid = 10 ** 6
step = 10000
cid_range = range(min_cid, max_cid, step)

for i in tqdm(cid_range):
    start_cid = i
    end_cid = i + step - 1
    insert_counts = 0
    # Starting
    logging.info("\n")
    logging.info(f"[+] Starting new process, from {start_cid} to {end_cid}!")
    working_dataset = preprocess_dataset(working_dataset=pubchem_dao.get_data_between_cid(start_cid, end_cid), train_test_dataset=train_test_dataset)
    for idx, row in working_dataset.iterrows():
        smiles = row['pubchem_openeye_iso_smiles']
        RDLogger.DisableLog('rdApp.info') 
        mol = Chem.MolFromSmiles(smiles)
        molecular_weight = ExactMolWt(mol)
        if molecular_weight <= 700:
            inserted = screening_dao.insert(cid=row['pubchem_compound_cid'], smiles=smiles, molecular_weight=molecular_weight)
            if inserted is True:
                insert_counts += 1
    # Output insert count
    logging.info("[+] Insert into database: " + str(insert_counts))

100%|██████████| 1/1 [00:00<00:00, 62.25it/s]


In [15]:
number_of_rows = screening_dao.get_number_of_rows()
number_of_rows

[(87263665,)]